In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_excel('/content/drive/MyDrive/[텍스트분석] 팀 프로젝트/dataset/article_df_final.xlsx', engine='openpyxl')
#test = pd.read_excel('/content/drive/MyDrive/[텍스트분석] 팀 프로젝트/dataset/data.xlsx', sheet_name='Sheet1', engine='openpyxl')
data.head(2)
#test_1=data[:1]

,Unnamed: 0,title,date,main,section,url
0,0,오후 국회 본회의…'전세사기 특별법'·'김남국 방지법' 처리,2023.05.25. 오전 9:24,전세사기 특별법과 이른바 '김남국 방지법'으로 불리는 가상자산(코인) 관련 개정안이...,100,https://n.news.naver.com/mnews/article/374/000...
1,1,"이준석, 학력 위조 의혹에 ""하버드 졸업 10억 내기하자…이번엔 박멸""",2023.05.25. 오후 3:53,"""해당 의혹 제기한 유튜버들 싸그리 고소""\n\n\n\n이준석 전 국민의힘 대표가 ...",100,https://n.news.naver.com/mnews/article/088/000...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def isUnique(item):#중복값 확인
    tempSet = set()
    return not any(i in tempSet or tempSet.add(i) for i in item)

In [ ]:
isUnique(data['main'])

True

In [ ]:
print('overview 열의 결측값의 수:',data['main'].isnull().sum())#결측값 확인

overview 열의 결측값의 수: 0


In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 39.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import re
import json
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
def preprocessing(review, okt, remove_stopwords = False, stop_words =[]):
  #함수인자설명
  # review: 전처리할 텍스트
  # okt: okt객체를 반복적으로 생성하지 않고 미리 생성 후 인자로 받음
  # remove_stopword: 불용어를 제거할지 여부 선택. 기본값 False
  # stop_words: 불용어 사전은 사용자가 직접 입력, 기본값 빈 리스트

  # 1. 한글 및 공백 제외한 문자 모두 제거
  review_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]','',review)

  #2. okt 객체를 활용하여 형태소 단어로 나눔
  word_review = okt.morphs(review_text,stem=True)

  if remove_stopwords:
    #3. 불용어 제거(선택)
    word_review = [token for token in word_review if not token in stop_words]
  return word_review

In [ ]:
stop_words = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한','올해','지난해','기준','최근','현재','회장','서비스','이후','오전','오후','대해','위해','내용','연합뉴스','때문','무단','따른','금지','전재','경향신문','기자','는데','파이낸셜','저작','뉴스','이번','대한','대비']
okt = Okt()
clean_train_review = []

for review in data['main']:
  # 리뷰가 문자열인 경우만 전처리 진행
  if type(review) == str:
    clean_train_review.append(preprocessing(review,okt,remove_stopwords=True,stop_words= stop_words))
  else:
    clean_train_review.append([]) #str이 아닌 행은 빈칸으로 놔두기

clean_train_review[:1]

KeyboardInterrupt: ignored

In [ ]:
#테스트 리뷰도 동일하게 전처리
#이 코드는 test데이터 사용할때만 돌리자!!

clean_test_review = []
for review in test:
  if type(review) == str:
    clean_test_review.append(preprocessing(review, okt, remove_stopwords=True, stop_words=stop_words))
  else:
    clean_test_review.append([])

NameError: ignored

In [ ]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(data['main'])
print('TF-IDF 행렬의 크기(shape) :',tfidf_matrix.shape)

TF-IDF 행렬의 크기(shape) : (330, 26372)


In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :',cosine_sim.shape)

코사인 유사도 연산 결과 : (330, 330)


In [ ]:
def get_recommendations(idx, cosine_sim=cosine_sim):
    # 해당 기사와 모든 기사와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 기사들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 20개의 기사를 받아온다.
    sim_scores = sim_scores[1:21]

    # 가장 유사한 20개의 기사의 인덱스를 얻는다.
    movie_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 20개의 기사내용과 유사도를 리턴한다.
    similar_articles = data[['main', 'section']].iloc[movie_indices]
    similar_articles['similarity'] = [idx[1] for idx in sim_scores]

    return similar_articles

In [ ]:
data[0:1]

,main,section
0,“간호법은 尹대통령 후보 시절 직접 약속한 법”\n\n\n\n박홍근 더불어민주당 원...,100


In [ ]:
goal = int(input("찾으려는 기사의 인덱스를 입력하세요: "))
print("해당 기사: ")
print(data.loc[goal])
print("\n유사 기사:")
ans = get_recommendations(goal)
ans

찾으려는 기사의 인덱스를 입력하세요: 10
해당 기사: 
main       국민의힘 김기현 대표가 13일 서울 여의도 국회에서 열린 최고위원회의에서 모두발언을...
section                                                  100
Name: 10, dtype: object

유사 기사:


,main,section,similarity
38,신임 원내대표 맞춰 주요 당직자 임명…당무감사위원장에 신의진\n\n\n\n행안부 '...,100,0.381573
121,"민주당 전현직 대표, 檢 수사선상 올라야권 인사 줄줄이 수사대상 포함될 듯내년 총선...",102,0.149558
8,"의원 20명 비례대표·선거구 등 해법 제시…""국회 불신 종식"" ""행정부 견제는 누가...",100,0.125705
17,"성역없는 엄정한 검찰수사 촉구…""송영길 사과하고 자진 조사 받아야""(서울=연합뉴스)...",100,0.104583
78,"중기중앙회, 이철우 경북도지사 초청 간담회 열어\n\n\n\n경북도 제공중소기업중앙...",101,0.104251
28,"""송영길, 돈봉투 오간 사실 모를 수 없어""\n\n\n\n국민의힘 김기현 대표가 1...",100,0.103076
145,"검찰, 의원 사무실 등 강제수사검찰이 12일 더불어민주당 전당대회에서 불법 정치자금...",102,0.099166
24,국민의힘 싱크탱크인 여의도연구원은 오늘(13일) 이사회를 열고 국민의힘 윤창현 의원...,100,0.094883
36,"검찰, 돈 전달 정황 담긴 녹취 확보... 윤관석 의원 ""검찰의 정치기획 수사""\n...",100,0.092243
173,중국발 황사가 한반도에 유입돼 전국 대부분 지역에 미세먼지경보 및 황사위기경보가 발...,103,0.090736


In [ ]:
print(data.loc[goal, 'section'],"\n")
ans['section'].value_counts()

100 



100    15
102     3
101     1
103     1
Name: section, dtype: int64

In [ ]:
type(ans)

pandas.core.frame.DataFrame